<h1 style="color:#4E2A84;">AWS - Braket - Rigetti - CNOT Crosstalk Example</h1>

This notebook provides example of creating a circuit with many $CNOT$ gates, assigning them to specific qubits and executing on the Rigetti Ankaa-3 quantum computer. The example circuit contained herein which many $CNOT$ gates does no useful computation, but rather can be used as example of a circuit that generates noise or crosstalk that affects other qubits on the quantum computer. Variants of circuits with many $CNOT$ gates have been used in research papers to demonstrate crosstalk attacks between two circuits executing at the same time on a quantum computer. To demonstrate this, the example circuit is augmented with one so-called victim qubit that his selected to be adjacent to the circuit with many $CNOT$ gates. The execution of the $CNOT$ gates causes crosstalk that is observed by analyzing how the victim's stat that should be in $\ket{0}$ changes to $\ket{1}$.

This notebook leverages the `AWS` cloud service, it uses the `Braket` development environment and targets a `Rigetti` superconducting quantum computer.

In [1]:
# reset all notebook variables just in case
%reset -f

<h1 style="color:#4E2A84;">Setup Amazon Braket Environment</h1> 

To begin, make sure that you have installed the latest Amazon Braket environment in qBraid Lab ([installation instructions](https://docs.qbraid.com/lab/user-guide/environments#install-environment)) and have selected the `Python [Braket vX.X]` kernel. (You can switch kernels by going to the Menu Bar → Kernel → Change Kernel). 

You can vew the kernel currently selected in the top-right of the editor window: <img src="images/image_select_kernel.png" alt="Kernel selection" style="width:200px;"/>

<h1 style="color:#4E2A84;">General Setup for Coding</h1> 

Now, you need to import the appropriate libraries. This notebook uses Braket development enviroment, which is initialized with the below imports.

In [2]:
from braket.tracking import Tracker
tracker = Tracker().start()

# general imports
import matplotlib.pyplot as plt

%matplotlib inline

# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit
from braket.devices import LocalSimulator
from braket.aws import AwsDevice, AwsQuantumTask

<h1 style="color:#4E2A84;">Partial Code for Circuit with Many CNOT Gates</h1>

Below cells specify a circuit with many $CNOT$ gates. Note the qubits used by the $CNOT$ gates have specific indices. The indices, i.e. the physical qubits used in the circuit, were selected specifically based on the topology of the Rigetti Ankaa-3 quantum computer. The topology is exemplified later in this notebook. For now we simply show how to create a circuit with $CNOT$ gates, targetting $10$ qubits.

Note that in Braket, we assign the physical (hardware) qubits when we define our circuit. It is different from IBM Qiskit, where the mapping of circuit qubits to hardware qubits is handled during the compilation step.

Below we will define two circuits, each with different arrangament of $CNOT$ gates. After that, the two circuits can be concatenated together to create an even bigger circuit, which will be the actual circuit we will use.

In [5]:
# define a circuit
circ_a = Circuit()

# add gates to the circuit
circ_a.cnot(77, 78)
circ_a.cnot(70, 71)
circ_a.cnot(63, 64)
circ_a.cnot(56, 57)
circ_a.cnot(49, 50)

Circuit('instructions': [Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(77), Qubit(78)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(70), Qubit(71)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(63), Qubit(64)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(56), Qubit(57)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(49), Qubit(50)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)])

In [6]:
# print the circuit for inspecction
print(circ_a)

T   : │  0  │
             
q49 : ───●───
         │   
       ┌─┴─┐ 
q50 : ─┤ X ├─
       └───┘ 
             
q56 : ───●───
         │   
       ┌─┴─┐ 
q57 : ─┤ X ├─
       └───┘ 
             
q63 : ───●───
         │   
       ┌─┴─┐ 
q64 : ─┤ X ├─
       └───┘ 
             
q70 : ───●───
         │   
       ┌─┴─┐ 
q71 : ─┤ X ├─
       └───┘ 
             
q77 : ───●───
         │   
       ┌─┴─┐ 
q78 : ─┤ X ├─
       └───┘ 
T   : │  0  │


In [7]:
# define a second circuit whith different arrangament of CNOT gates
circ_b = Circuit()

# add gates to the circuit
circ_b.cnot(78, 71)
circ_b.cnot(70, 63)
circ_b.cnot(64, 57)
circ_b.cnot(56, 49)

Circuit('instructions': [Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(78), Qubit(71)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(70), Qubit(63)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(64), Qubit(57)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(56), Qubit(49)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)])

In [9]:
# print the circuit for inspecction
print(circ_b)

T   : │     0     │
       ┌───┐       
q49 : ─┤ X ├───────
       └─┬─┘       
         │         
q56 : ───●─────────
                   
             ┌───┐ 
q57 : ───────┤ X ├─
             └─┬─┘ 
       ┌───┐   │   
q63 : ─┤ X ├───┼───
       └─┬─┘   │   
         │     │   
q64 : ───┼─────●───
         │         
         │         
q70 : ───●─────────
                   
       ┌───┐       
q71 : ─┤ X ├───────
       └─┬─┘       
         │         
q78 : ───●─────────
                   
T   : │     0     │


<h1 style="color:#4E2A84;">Put Together Code for a Circuit with Many CNOT Gates</h1>

Now we will combine the smaller circuits, $circ\_a$ and $circ\_b$ to create a bigger circuit. Althought the circuit does no useful work and the outputs will not make sense, we will add the $circ\_m$ circuit with just measurements at the end of the combined circuit so that the overall bigger circuit has proper measurment at end on all the qubits.

You can adjust the $num\_copies$ variable to control how many copies of the circuits will be concatenated together.

In [18]:
# create the main circuit
circ_many_cnot_gates = Circuit()

# set now many copies of the circuits to concatenate together
num_copies = 2

# use a loop to concatenate many copies of the circuits together
for n in range(0,num_copies):
    circ_many_cnot_gates += circ_a
    circ_many_cnot_gates += circ_b

In [19]:
# print the circuit
print(circ_many_cnot_gates)

T   : │  0  │     1     │  2  │     3     │
             ┌───┐             ┌───┐       
q49 : ───●───┤ X ├─────────●───┤ X ├───────
         │   └─┬─┘         │   └─┬─┘       
       ┌─┴─┐   │         ┌─┴─┐   │         
q50 : ─┤ X ├───┼─────────┤ X ├───┼─────────
       └───┘   │         └───┘   │         
               │                 │         
q56 : ───●─────●───────────●─────●─────────
         │                 │               
       ┌─┴─┐       ┌───┐ ┌─┴─┐       ┌───┐ 
q57 : ─┤ X ├───────┤ X ├─┤ X ├───────┤ X ├─
       └───┘       └─┬─┘ └───┘       └─┬─┘ 
             ┌───┐   │         ┌───┐   │   
q63 : ───●───┤ X ├───┼─────●───┤ X ├───┼───
         │   └─┬─┘   │     │   └─┬─┘   │   
       ┌─┴─┐   │     │   ┌─┴─┐   │     │   
q64 : ─┤ X ├───┼─────●───┤ X ├───┼─────●───
       └───┘   │         └───┘   │         
               │                 │         
q70 : ───●─────●───────────●─────●─────────
         │                 │               
       ┌─┴─┐ ┌───┐       ┌─┴─┐ ┌

<h1 style="color:#4E2A84;">Add Simple Victim Qubit and a Reference Qubit</h1>

To evaluate the effect of many $CNOT$ gates, we need a so-called victim qubit. We will select a qubit that is adjacent to the circuit with the $CNOT$ gates. Here the victim qubit will be qubit $65$.

To compare the potential results of the crosstalk to behavior of an unaffected qubit, we need to have a reference qubit somewhere else in the topology. We will select qubit $6$ as the reference qubit, it is far away from the qubits where $CNOT$ gates execute, and so it should not be impacted by the crosstalk.

The victim's location and the reference qubit's location can be visualized on the Ankaa-3 topology:

<img src="images/cnot_crosstalk_example_on_ankaa_3_topology.png" alt="Kernel selection" style="width:500px;"/>

In [22]:
# new circuit for the victim qubit
victim_qubit_index = 65
circ_victim = Circuit()

# new circuit for a reference qubit
reference_qubit_index = 6
circ_reference = Circuit()

# put all the circuits together
circ_cnots_and_victim = Circuit()
circ_cnots_and_victim += circ_many_cnot_gates
circ_cnots_and_victim += circ_victim
circ_cnots_and_victim += circ_reference

In [23]:
# print circuit for visual inspection
print(circ_cnots_and_victim)

T   : │  0  │     1     │  2  │     3     │
             ┌───┐             ┌───┐       
q49 : ───●───┤ X ├─────────●───┤ X ├───────
         │   └─┬─┘         │   └─┬─┘       
       ┌─┴─┐   │         ┌─┴─┐   │         
q50 : ─┤ X ├───┼─────────┤ X ├───┼─────────
       └───┘   │         └───┘   │         
               │                 │         
q56 : ───●─────●───────────●─────●─────────
         │                 │               
       ┌─┴─┐       ┌───┐ ┌─┴─┐       ┌───┐ 
q57 : ─┤ X ├───────┤ X ├─┤ X ├───────┤ X ├─
       └───┘       └─┬─┘ └───┘       └─┬─┘ 
             ┌───┐   │         ┌───┐   │   
q63 : ───●───┤ X ├───┼─────●───┤ X ├───┼───
         │   └─┬─┘   │     │   └─┬─┘   │   
       ┌─┴─┐   │     │   ┌─┴─┐   │     │   
q64 : ─┤ X ├───┼─────●───┤ X ├───┼─────●───
       └───┘   │         └───┘   │         
               │                 │         
q70 : ───●─────●───────────●─────●─────────
         │                 │               
       ┌─┴─┐ ┌───┐       ┌─┴─┐ ┌

<h1 style="color:#4E2A84;">Add Measurement to All Qubits</h1>

With the circuit completed, the last part is now to add measurement to all the qubits.

In [24]:
# add final measurement to all qubits
circ_cnots_and_victim.measure([49,50,56,57,63,64,70,71,77,78,66,6])

Circuit('instructions': [Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(77), Qubit(78)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(70), Qubit(71)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(63), Qubit(64)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(56), Qubit(57)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(49), Qubit(50)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(78), Qubit(71)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Q

In [26]:
# print circuit for inspection
print(circ_cnots_and_victim)

T   : │  0  │     1     │  2  │     3     │  4  │
                                           ┌───┐ 
q6  : ─────────────────────────────────────┤ M ├─
                                           └───┘ 
             ┌───┐             ┌───┐       ┌───┐ 
q49 : ───●───┤ X ├─────────●───┤ X ├───────┤ M ├─
         │   └─┬─┘         │   └─┬─┘       └───┘ 
       ┌─┴─┐   │         ┌─┴─┐   │         ┌───┐ 
q50 : ─┤ X ├───┼─────────┤ X ├───┼─────────┤ M ├─
       └───┘   │         └───┘   │         └───┘ 
               │                 │         ┌───┐ 
q56 : ───●─────●───────────●─────●─────────┤ M ├─
         │                 │               └───┘ 
       ┌─┴─┐       ┌───┐ ┌─┴─┐       ┌───┐ ┌───┐ 
q57 : ─┤ X ├───────┤ X ├─┤ X ├───────┤ X ├─┤ M ├─
       └───┘       └─┬─┘ └───┘       └─┬─┘ └───┘ 
             ┌───┐   │         ┌───┐   │   ┌───┐ 
q63 : ───●───┤ X ├───┼─────●───┤ X ├───┼───┤ M ├─
         │   └─┬─┘   │     │   └─┬─┘   │   └───┘ 
       ┌─┴─┐   │     │   ┌─┴─┐   │     │   ┌───┐ 


<h1 style="color:#4E2A84;">Estimate the Circuit Duration</h1>

As a minor sanity check, we want to check the duration of the circuit we generated. It is important to note that the qubits decohere, and computation cannot usually run longer than the so called $T1$ time ($T1$ time is an average, so in some cases compuation can run slightly longer than $T1$). If the circuit with many $CNOT$ gates were used to generate crosstalk and evaluate potentail qubit flits, the circuit with many $CNOT$ gates has to execute in the time shorter than $T1$. Otherwise, any qubit flips or state changes in other qubits affected by the crosstalk may actually be due to decoherence and not crosstalk.

In summary, when running any experiments or circuits that are used to generate and evaluate crosstalk, all the circuits have to finish execution in time less than $T1$ so that the qubit flips can be attribtued to crosstalk.

In [28]:
# circuit to estimate
circ_to_estimate = circ_cnots_and_victim

# typical approximate parameters for Ankaa-3, time units are ns
typical_parameters_ankaa_3_ns = {
    "t1": 22000,
    "t2": 19000,
    "cnot": 72,
}

# get depth of the circuit
print("Gate depth:", circ_to_estimate.depth)

# sanity check
if circ_to_estimate.depth * typical_parameters_ankaa_3_ns['cnot'] < typical_parameters_ankaa_3_ns['t1']:
    print("Based on gate depth, total circuit execution time seems to be less than the decoherence time.")
else:
    print("Based on gate depth, total circuit execution time seems to be more than the decoherence time.")
    print("Be careful that any experiment results are affected by decoherence!")


Gate depth: 5
Based on gate depth, total circuit execution time seems to be less than the decoherence time.


<h1 style="color:#4E2A84;">Skip Simulation</h1> 

Simple state-vector simulators do not simulate crosstalk, thus running circuits used for generating crosstalk, or trying to evalute crosstalk in state-vector simulators is not helpful. We will directly proceed to estimate the execution cost and run the circuit on real quantum computer.

<h1 style="color:#4E2A84;">Set Number of Shots</h1> 

Before estimating the cost and executing the circuit, we need to specify the number of shots. As usually, a number aroung $1000$ should be sufficient.

In [ ]:
# set number of shots
circuit_shots = 1000

<h1 style="color:#4E2A84;">Estimate qBraids Credits Needed</h1> 

The cost to run a circuit on a quantum computer is divided into per-task cost (effectively the cost to submit a circuit for execution) and per-shot cost (additional cost for each shot).

You can view the cost by selecting $DEVICES$ on the right-hand side of the editor, then search for Rigetti Ankaa-3 (AWS) and finally selecting the pricing cell. <img src="images/image_pricing.png" alt="View pricing" style="width:300px;"/>

The current cost is $30$ qBraid Credits per-task on Ankaa-3 and then $0.09$ qBraid Credits per-task.

Currently each qBraid credit is valued at $0.01$ USD. This means that $100$ credits equate to $1.00$ USD.

In [ ]:
# compute cost
perTaksCost = 30.00
perShotCost = 0.09
totalCost = perTaksCost+perShotCost*circuit_shots
totalCostDollars = totalCost/100

# print cost
print(f'Cost to run Bell circuit on Rigetti Ankaa-3 via AWS is:')
print(f'{totalCost} qBraid Credit or equivalently ${totalCostDollars} USD')

<h1 style="color:#4E2A84;">Request qBraid Credits</h1> 

If you do not have AWS credentials for submitting your quantum jobs, you will need qBraid Credits to run the code on actual quantum computer. Please request qBraid Credits from the instructor. After you have qBraid Credits then you can proceed with the following code.

<h1 style="color:#4E2A84;">Submit the Circuit to Rigetti Ankaa-3 and Obtain the Results</h1>

You can now submit the circuit for execution.

Please note, we’ve disabled qubit rewiring to allow us to ensure the qubits numbers we used in the circuit definition are exactly the ones which be used to execute our circuit. E.g. if the victim is set to qubit $65$, with qubit rewriting disabled, the complier will not change or move our victim qubit. For Rigetti, qubit rewiring must be turned off by setting disableQubitRewiring=True for use with verbatim compilation. If disableQubitRewiring=False is set when using verbatim boxes in a compilation, the quantum circuit fails validation and does not run.

In [ ]:
# circuit to run
circ_to_run = circ_cnots_and_victim

# print circuit
print(circ_to_run)

In [ ]:
# set up device
ankaa = AwsDevice('arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3')

# run circuit
ankaa_task = ankaa.run(circ_to_run, shots=circuit_shots, disable_qubit_rewiring=True)
    
# get id and status of submitted task
ankaa_task_id = ankaa_task.id
ankaa_status = ankaa_task.state()

# print task status
print(f'Status of task: {ankaa_status}\n')

# print task id for future reference
print(f'Task id: {ankaa_task_id}')

In [ ]:
# get task arn
ankaa_task_arn = ''

# load the quantum task
ankaa_task_id = AwsQuantumTask(arn=ankaa_task_arn)

# print status
status = ankaa_task_id.state()
print("Status of the task:", status)

print(ankaa_task_id.state())
print(ankaa_task_id.metadata())  # Includes status, failure reason

<h1 style="color:#4E2A84;">Analyze Results</h1>



In [ ]:
# get results
ankaa_results = ankaa_task_id.result()

# get measurement counts
ankaa_counts = ankaa_results.measurement_counts
print("Measurement counts:", ankaa_counts)